In [4]:
from connectN import *

In [261]:
import pandas as pd
import random
import copy
import math

In [262]:
EXP_PARAMETER = math.sqrt(2)

In [722]:
#  Here is my implementation of the game_node class
# a game_node is meant to represent a node in our monte carlo tree
class game_node:
    def __init__(self, game, parent,prior_move):
        #Who's your daddy?  A pointer to the parent.  Should be None if the root.
        self.parent = parent
        
        #How many times have you been here?  Did you win?  Currently not used.
        self.num_visits = 0
        self.num_wins = 0
        #We store the current board state.  Ultimately unnecessary but easy for now
        self.game = game
        #What are the legal moves and how many
        self.legal_moves = game.legal_moves
        self.num_legal_moves = len(self.legal_moves)
        
        # We store the move made to arrive to the current game node.  Useful for backpropagation.  Possibly not needed but not harmful
        self.prior_move = prior_move
        
        # the statistics about children are stores so that we can pick which child to visit when we are here
        #child_scores scored as a pandas dataframe with 3 columns: num_visits, num_wins, score
        #score is only updated once all children have been visited
        #of note:  this is likely the most wasteful part, especially since if we replace it with a numpy array, we can
        #probably use Numba to speed up our code tremendously
        self.child_scores =pd.DataFrame(dtype = 'float',index = game.legal_moves, columns = ['num_visits','num_wins','score']).fillna(0)
        
        # children is a dictionary with keys given by legal_moves and outputs are pointers to the appropriate children
        #it is updated each time a new child is visited
        self.children = {}
        #Active player isn't strictly necessary, but so easy to store.
        self.active_player = (-1)**(game.turn)

        # This should be edited to make it so that if a parent has a winning move, we stop doing any thinking there.  why bother?
        # For now it stays, but in future updates it should be gone.
        if game.is_won:
            self.all_children_visited = True
        else:
            self.all_children_visited = False

        
    #pick_and_play_move picks the best new move as per score if all children visited
    # if there remains an unvisited child, then one is picked at random
    # The output is [pointer_to_child_node, Boolean_value]
    # the boolean is set to true if the child is a newly created node (and thus we should play a random game)

    def pick_and_play_move(self):
        if self.game.is_won:
            print('Make a better error code')
            return(None)
        if self.all_children_visited:
            next_move = self.child_scores.score.idxmax()
            return([self.children[next_move],  False])
        else:
            df = self.child_scores
            moves_left = df[df.num_visits ==0].index.values
            choice = random.randint(0,len(moves_left)-1)
            return([self.play_new_move(moves_left[choice]), True])

    
    def play_random_move(self):
        n = len(self.legal_moves)
        return(self.play_new_move(self.legal_moves[random.randint(0,n-1)]))

    # play_new_move creates a child node using the move legal_move updates our internal data and then returns a pointer to the child    
    def play_new_move(self, legal_move):
        child_game = copy.deepcopy(self.game)
        child_game.play_chip(legal_move)
        child_node = game_node(child_game, self,legal_move)
        self.children[legal_move] = child_node
        return(child_node)
        

    # backpropogate is probably misspelled.  It accepts as input the player who won the pertinent leaf (1 or -1)
    # the top_node to which we wish to backpropogate (no sense going back to the root if we are on move 20)
    # and the last move_played (so that the parent can easily update its internal statistics)
    # it updates the current node's child_scores and then recursively calls itself on the parent.  the return is the parent
    
    def backpropogate(self, winner, top_node = None, move_played = None):
        self.num_visits = self.num_visits+1
        score = ((-1)**(self.game.turn)*winner+1)/2
        self.update_child_scores(score = score, move_played = move_played)
        if (top_node == self) or (self.parent==None):
            return(self)
        else:
            return(self.parent.backpropogate( winner = winner, top_node = top_node, move_played = self.prior_move))
    
    # update_child_scores takes as input the score (-1 or 1 if one of them wins, 0 if it's a draw) and the move that
    #was played in this position
    # it then updates our child_scores dataframe
    def update_child_scores(self, score, move_played):

        df = self.child_scores
        df.loc[move_played,'num_visits'] = df.loc[move_played,'num_visits']+1
        df.loc[move_played,'num_wins'] = df.loc[move_played,'num_wins']+score
        if self.all_children_visited:
            df['score'] = df.num_wins/df.num_visits+EXP_PARAMETER*np.sqrt(np.log(self.num_visits)/df.num_visits)
        elif not (df.num_visits==0).sum():
            self.all_children_visited=True

                

    

In [728]:
##########  Other Scripts
# These are functions we call from outside game nodes.  Eventually it might make sense to have
# a game_tree class that manages them
#play_random_game(node) plays a random game starting at the given node and returns the final position.
def play_random_game(node):
    current_node = node
    while not current_node.game.is_won:
        if current_node.num_legal_moves ==0:
            return(current_node)
        else:
            current_node = current_node.play_random_move()
    return(current_node)

#simulate_round starts at the input node and explores until it sees a new vertex created
#then it plays a random game and backpropogates the score
def simulate_round(node):
    current_node = node
    flag = False
    while not flag | current_node.game.is_won:
        [current_node,flag] = current_node.pick_and_play_move()
    if current_node.game.is_won:
        outcome = current_node
    else:
        outcome = play_random_game(new_node)
    if outcome.game.is_won:
        winner = (-1)**(outcome.game.turn+1)
    else:
        winner = 0
        print('good game! it\'s a draw!')
    outcome.parent.backpropogate(winner = winner, top_node = node, move_played = outcome.prior_move)
        

In [753]:
root = game_node(game, None, None)

In [762]:
for i in range(5000):
    current_node = root
    flag = False
    while not flag | current_node.game.is_won:
        [current_node,flag] = current_node.pick_and_play_move()
    if current_node.game.is_won:
        outcome = current_node
    else:
#         new_node = current_node.pick_and_play_move()[0]
        outcome = play_random_game(current_node)
    if outcome.game.is_won:
        winner = (-1)**(outcome.game.turn+1)
    else:
        winner = 0
        print('good game! it\'s a draw!')
    outcome.parent.backpropogate(winner = winner, top_node = root, move_played = outcome.prior_move)

good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!
good game! it's a draw!


In [763]:
first_move = root.child_scores.copy()
first_move['win_rates'] = first_move.num_wins/first_move.num_visits

In [764]:
first_move

,num_visits,num_wins,score,win_rates
0,338.0,167.5,0.721309,0.495562
1,475.0,252.5,0.722008,0.531579
2,434.0,226.5,0.721110,0.521889
3,2561.0,1646.0,0.724729,0.642718
4,771.0,441.5,0.722102,0.572633
5,691.0,389.5,0.721561,0.563676
6,230.0,103.0,0.721489,0.447826


In [769]:
second_move = root.children[3].child_scores.copy()
second_move['win_rates'] = second_move.num_wins/second_move.num_visits
second_move

,num_visits,num_wins,score,win_rates
0,232.0,70.0,0.561833,0.301724
1,405.0,147.5,0.561064,0.364198
2,439.0,163.5,0.561526,0.372437
3,484.0,184.5,0.561283,0.381198
4,538.0,210.0,0.561142,0.390335
5,263.0,83.5,0.561789,0.317490
6,200.0,56.0,0.560146,0.280000
